<a href="https://colab.research.google.com/github/AOMDiputacio/Scrapper/blob/main/Generar_tablas_de_precios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A partir del archivo '/content/drive/MyDrive/Scrapper-Pruebadecolchones/url_nombre_a_scrappear.csv'


#Ikea

In [121]:
from bs4 import BeautifulSoup
import requests as r
import pandas as pd
import json
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive', force_remount=True)

# Rutas de los archivos en Google Drive
ruta_archivo_csv = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/url_nombre_a_scrappear.csv'

# Leer el CSV desde Google Drive con especificación de la codificación
df_urls = pd.read_csv(ruta_archivo_csv, encoding='latin1')

# Check for NaN values in the 'URL' column
df_urls = df_urls.dropna(subset=['URL'])

# Filtrar las filas que contienen 'ikea.com' en la columna 'url'
df_urls = df_urls[df_urls['URL'].str.contains('ikea.com')]

# Obtener la lista de URLs, nombres, y variantes desde el DataFrame
urles = df_urls['URL'].tolist()
nombres = df_urls['Nombre'].tolist()

# Inicializar la lista json_list
json_list = []

# Iterar sobre las URLs y obtener información
for url, nombre in zip(urles, nombres):
    # Conseguir URL por dimensión por el modelo seleccionado

    try:
        reql = r.get(url)
        reql.raise_for_status()  # Verificar si hay errores en la solicitud HTTP
        soup = BeautifulSoup(reql.text, "html.parser")
        urldim = soup.find_all('div', class_='js-product-variation-section pip-product-variation-section')

        for div in urldim:
            data_initial_props = div['data-initial-props']
            json_data = json.loads(data_initial_props)

            # Obtener información del precio anidado
            for option in json_data['variations'][0]['options']:
                price_props = option['priceProps']
                precio_original = price_props.get('priceNumeral')
                dimension = option['title']

                # Agregar a la lista si priceNumeral está presente
                if precio_original is not None:
                    json_list.append({'nombre': nombre, 'dimension': dimension, 'precio_original': precio_original, 'precio_con_descuento': precio_original})

    except Exception as e:
        print(f"Error al procesar la URL: {url}")
        print(f"Error: {e}")

# Crear un DataFrame con la información de "priceNumeral" y "dimension"
matriz_precios_ikea = pd.DataFrame(json_list, columns=['nombre', 'dimension', 'precio_original', 'precio_con_descuento'])

# Mostrar el DataFrame
print(matriz_precios_ikea)



Mounted at /content/drive
           nombre   dimension  precio_original  precio_con_descuento
0    IKEA Valevag   80x200 cm              199                   199
1    IKEA Valevag   90x190 cm              199                   199
2    IKEA Valevag   90x200 cm              199                   199
3    IKEA Valevag  105x190 cm              269                   269
4    IKEA Valevag  135x190 cm              299                   299
5    IKEA Valevag  140x200 cm              299                   299
6    IKEA Valevag  150x190 cm              369                   369
7    IKEA Valevag  160x200 cm              369                   369
8    IKEA Valevag  180x200 cm              459                   459
9   IKEA Vesteroy   80x200 cm              169                   169
10  IKEA Vesteroy   90x190 cm              169                   169
11  IKEA Vesteroy   90x200 cm              169                   169
12  IKEA Vesteroy  105x190 cm              199                   199
13  IKEA

In [109]:
from bs4 import BeautifulSoup
import requests as r
import pandas as pd
import json
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive', force_remount=True)

# Rutas de los archivos en Google Drive
ruta_archivo_csv = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/url_nombre_a_scrappear.csv'

# Leer el CSV desde Google Drive con especificación de la codificación
df_urls = pd.read_csv(ruta_archivo_csv, encoding='latin1')

# Check for NaN values in the 'URL' column
df_urls = df_urls.dropna(subset=['URL'])

# Filtrar las filas que contienen 'ikea.com' en la columna 'url'
df_urls = df_urls[df_urls['URL'].str.contains('ikea.com')]

# Obtener la lista de URLs, nombres, y variantes desde el DataFrame
urles = df_urls['URL'].tolist()
nombres = df_urls['Nombre'].tolist()

# Inicializar la lista json_list
json_list = []

# Iterar sobre las URLs y obtener información
for url, nombre in zip(urles, nombres):
    # Conseguir URL por dimensión por el modelo seleccionado
    url = url + "#content"

    try:
        reql = r.get(url)
        reql.raise_for_status()  # Verificar si hay errores en la solicitud HTTP
        soup = BeautifulSoup(reql.text, "html.parser")
        urldim = soup.find_all('div', class_='js-product-variation-section pip-product-variation-section')

        for div in urldim:
            data_initial_props = div['data-initial-props']
            json_data = json.loads(data_initial_props)

            # Obtener información del precio anidado
            for option in json_data['variations'][0]['options']:
                price_props = option['priceProps']
                precio_original = price_props.get('priceNumeral')
                title = option['title']

                # Agregar a la lista si priceNumeral está presente
                if precio_original is not None:
                    json_list.append({'priceNumeral': precio_original, 'title': title})

    except Exception as e:
        print(f"Error al procesar la URL: {url}")
        print(f"Error: {e}")

# Crear un DataFrame con la información de "priceNumeral" y "title"
df = pd.DataFrame(json_list, columns=['title','priceNumeral','priceNumeral' ])

# Mostrar el DataFrame
print(df)

Mounted at /content/drive
         title  priceNumeral  priceNumeral
0    80x200 cm           199           199
1    90x190 cm           199           199
2    90x200 cm           199           199
3   105x190 cm           269           269
4   135x190 cm           299           299
5   140x200 cm           299           299
6   150x190 cm           369           369
7   160x200 cm           369           369
8   180x200 cm           459           459
9    80x200 cm           169           169
10   90x190 cm           169           169
11   90x200 cm           169           169
12  105x190 cm           199           199
13  135x190 cm           199           199
14  140x200 cm           199           199
15  150x190 cm           299           299
16  160x200 cm           299           299


In [110]:
from bs4 import BeautifulSoup
import requests as r
import pandas as pd
import json
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive', force_remount=True)

# Rutas de los archivos en Google Drive
ruta_archivo_csv = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/url_nombre_a_scrappear.csv'

# Leer el CSV desde Google Drive con especificación de la codificación
df_urls = pd.read_csv(ruta_archivo_csv, encoding='latin1')

# Check for NaN values in the 'URL' column
df_urls = df_urls.dropna(subset=['URL'])

# Filtrar las filas que contienen 'hypnia.es' en la columna 'url'
df_urls = df_urls[df_urls['URL'].str.contains('ikea.com')]

# Obtener la lista de URLs, nombres, y variantes desde el DataFrame
urles = df_urls['URL'].tolist()
nombres = df_urls['Nombre'].tolist()

print(urles)
print(nombres)

url=urles[0]
nombre=nombres[0]

##Conseguir URL por dimensión por el modelo seleccionado
url = url + "#content"
reql = r.get(url)
soup = BeautifulSoup(reql.text, "html.parser")
urldim = soup.find_all('div', class_='js-product-variation-section pip-product-variation-section')
for div in urldim:
    data_initial_props = div['data-initial-props']
    json_data = json.loads(data_initial_props)
    json_list.extend(json_data['variations'][0]['options'])

# Crear un DataFrame con la información de "priceNumeral" y "title"
df = pd.DataFrame(json_list, columns=['priceNumeral', 'title'])

# Mostrar el DataFrame
print(df)







Mounted at /content/drive
['https://www.ikea.com/es/es/p/valevag-colchon-muelles-ensacados-firme-azul-claro-80531731/', 'https://www.ikea.com/es/es/p/vesteroy-colchon-muelles-ensacados-firme-azul-claro-30450609/']
['IKEA Valevag', 'IKEA Vesteroy']
    priceNumeral       title
0          199.0   80x200 cm
1          199.0   90x190 cm
2          199.0   90x200 cm
3          269.0  105x190 cm
4          299.0  135x190 cm
5          299.0  140x200 cm
6          369.0  150x190 cm
7          369.0  160x200 cm
8          459.0  180x200 cm
9          169.0   80x200 cm
10         169.0   90x190 cm
11         169.0   90x200 cm
12         199.0  105x190 cm
13         199.0  135x190 cm
14         199.0  140x200 cm
15         299.0  150x190 cm
16         299.0  160x200 cm
17           NaN   80x200 cm
18           NaN   90x190 cm
19           NaN   90x200 cm
20           NaN  105x190 cm
21           NaN  135x190 cm
22           NaN  140x200 cm
23           NaN  150x190 cm
24           NaN  160x200 c

#Hypnia

In [111]:
from bs4 import BeautifulSoup
import requests as r
import pandas as pd
import json
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive', force_remount=True)

# Rutas de los archivos en Google Drive
ruta_archivo_csv = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/url_nombre_a_scrappear.csv'

# Leer el CSV desde Google Drive con especificación de la codificación
df_urls = pd.read_csv(ruta_archivo_csv, encoding='latin1')

# Check for NaN values in the 'URL' column
df_urls = df_urls.dropna(subset=['URL'])

# Filtrar las filas que contienen 'hypnia.es' en la columna 'url'
df_urls = df_urls[df_urls['URL'].str.contains('hypnia.es')]

# Obtener la lista de URLs, nombres, y variantes desde el DataFrame
urles = df_urls['URL'].tolist()
nombres = df_urls['Nombre'].tolist()
urles = [urle + "?variant=" for urle in urles]

# Crear una lista para almacenar los DataFrames de cada URL
dfs = []

# Iterar sobre las URLs
for urle, nombre in zip(urles, nombres):  # Iterate over both URL and nombre simultaneously

    # Función para obtener precios desde la URL
    def obtener_precios(id):
        url = f"{urle}{id}"
        reql = r.get(url)
        soup = BeautifulSoup(reql.text, "html.parser")
        precios = soup.find('div', class_='product_price')

        if precios:
            precio_original = precios.find('s').text if precios.find('s') else None
            precio_con_descuento = precios.find_all('span')[1].text if len(precios.find_all('span')) > 1 else None
            return precio_original, precio_con_descuento
        else:
            return None, None

    # Obtener el contenido de la página
    reql = r.get(urle)
    soup = BeautifulSoup(reql.text, "html.parser")

    # Encontrar todos los scripts con type 'application/json'
    scripts = soup.find_all('script', type='application/json')

    # Iterar sobre los scripts y encontrar el que contiene datos relevantes
    for script in scripts:
        if "name" in script.string:
            schema = script.string
            break

    # Extraer datos del JSON
    data = json.loads(schema)

    # Crear DataFrame
    df = pd.DataFrame(data, columns=["name", "id", "title", "price"])

    # Renombrar columnas
    df.columns = ["name", "id", "title", "price"]

    # Agregar una columna 'nombre' al DataFrame
    df['nombre'] = nombre

    # Iterar sobre cada fila del DataFrame y añadir las columnas
    for index, row in df.iterrows():
        id_variant = row['id']
        precio_original, precio_con_descuento = obtener_precios(id_variant)
        df.at[index, 'precio_original'] = precio_original
        df.at[index, 'precio_con_descuento'] = precio_con_descuento

    # Agregar el DataFrame a la lista
    dfs.append(df)

# Concatenar todos los DataFrames en uno solo
final_df = pd.concat(dfs, ignore_index=True)



# Formatear tabla y columnas

# Definir una función para formatear precios y limpiar título
def formatear_datos(row):
    # Formatear precios
    for columna_precio in ['precio_original', 'precio_con_descuento']:
        if pd.notna(row[columna_precio]):
            # Eliminar el símbolo € y el punto de miles, luego reemplazar la coma decimal por un punto y eliminar espacios
            row[columna_precio] = str(row[columna_precio]).replace('€', '').replace('.', '').replace(' ', '')
            # Agregar el punto decimal para indicar decimales
            row[columna_precio] = row[columna_precio].replace(',', '.', 1)

    # Limpiar título y suprimir espacios
    if ' (cm)' in row['title']:
        row['title'] = row['title'].split(' (cm)')[0].replace(' ', '')

    return row

# Aplicar la función a las filas del DataFrame
final_df = final_df.apply(formatear_datos, axis=1)

#cambiar nombre de columna
final_df.rename(columns={'title': 'dimension'}, inplace=True)

# Mostrar solo las últimas tres columnas del DataFrame
#print(final_df[['nombre', 'label', 'precio_original', 'precio_con_descuento']])


# Crear un nuevo DataFrame con las columnas deseadas
matriz_precios_hypnia = final_df[['nombre', 'dimension', 'precio_original', 'precio_con_descuento']]

# Imprimir el resultado
print(matriz_precios_hypnia)

# Subir resultados en Google drive

# Rutas de los archivos en Google Drive
ruta_archivo_json = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/matriz_hypnia.json'
ruta_archivo_csv = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/matriz_hypnia.csv'

# Guardar el DataFrame en un archivo JSON en Google Drive
#matriz_precios_hypnia.to_json(ruta_archivo_json, orient='records', lines=True)

# Guardar el DataFrame en un archivo CSV en Google Drive
#matriz_precios_hypnia.to_csv(ruta_archivo_csv, index=False)


Mounted at /content/drive
                            nombre        dimension precio_original  \
0    Almohada Efecto Plumas Hypnia  40 x 70 cm (x2)           99.00   
1       Almohada Ergonómica Hypnia    56 x 36 cm x2          128.00   
2       Almohada Ergonómica Hypnia       56 x 36 cm           70.00   
3          Almohada Luna de Hypnia            50x70           55.00   
4          Almohada Luna de Hypnia            50x70           99.00   
..                             ...              ...             ...   
152       Topper Premium de Hypnia          160x200          375.00   
153       Topper Premium de Hypnia          180x200          425.00   
154                Hypnia Esencial           90x190             NaN   
155                Hypnia Esencial          160x200             NaN   
156                Hypnia Esencial          180x200             NaN   

    precio_con_descuento  
0                  49.00  
1                  89.00  
2                  49.00  
3            

#Pikolin


In [112]:
from bs4 import BeautifulSoup
import requests as r
import pandas as pd
import json
from google.colab import drive
import re

# Montar Google Drive
drive.mount('/content/drive', force_remount=True)

# Rutas de los archivos en Google Drive
ruta_archivo_csv = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/url_nombre_a_scrappear.csv'

# Leer el CSV desde Google Drive con especificación de la codificación
df_urls = pd.read_csv(ruta_archivo_csv, encoding='latin1')

# Check for NaN values in the 'URL' column
df_urls = df_urls.dropna(subset=['URL'])

# Filtrar las filas que contienen 'pikolin.com' en la columna 'URL'
df_urls_pikolin = df_urls[df_urls['URL'].str.contains('pikolin.com')]

# Inicializar una lista vacía para almacenar los resultados de cada fila
results_list = []

def extract_json_objects(script_tags):
    json_objects = []
    for script_tag in script_tags:
        try:
            if script_tag.string:
                json_content = json.loads(script_tag.string)
                json_objects.append(json_content)

                pattern = r'"defaultProductId": (\d+)'
                match = re.search(pattern, script_tag.string)

                if match:
                    default_product_id = match.group(1)
                    target_product_id = default_product_id

        except json.JSONDecodeError as e:
            print(f"Error al decodificar JSON en la etiqueta de script:\n{script_tag}\nError: {e}")

    return json_objects, target_product_id

def get_product_data(json_content, target_product_id):
    product_id_data = json_content.get(f"[product-id='{target_product_id}']", {}).get("Magento_Swatches/js/swatch-renderer", {}).get("jsonConfig", {}).get("attributes", {})
    return product_id_data

def create_table(data, column_names):
    table = pd.DataFrame(data, columns=column_names)
    return table

# Iterar a través de todas las filas en df_urls_pikolin
for index, row in df_urls_pikolin.iterrows():
    url = row['URL']
    nombre = row['Nombre']

    # Realizar una solicitud para la URL actual
    response = r.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Encontrar todas las etiquetas de script
    script_tags = soup.find_all('script', {'type': 'text/x-magento-init'})

    # Extraer objetos JSON de las etiquetas de script
    json_objects, target_product_id = extract_json_objects(script_tags)

    # Extract JSON objects from script tags
    for script_tag in script_tags:
        try:
            # Check if the 'string' attribute is not None
            if script_tag.string:
                # Check if the script tag contains the target product ID
                if f'"[product-id=\'{target_product_id}\']":' in script_tag.string:
                    # Parse the text content of the script tag as JSON
                    json_content = json.loads(script_tag.string)

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in script tag:\n{script_tag}\nError: {e}")

    # Obtener datos relacionados con el producto usando el ID del producto
    product_id_data = get_product_data(json_content, target_product_id)

    # Crear una tabla con columnas 'Id', 'label' y 'products'
    column_names1 = ['Id', 'label', 'products']
    table1_data = [{'Id': option['id'], 'label': option['label'], 'products': ', '.join(option['products'])} for option in product_id_data.get('272', {}).get('options', [])]
    df_table1 = create_table(table1_data, column_names1)

    # Obtener datos de precios de opciones
    option_prices_data = json_content.get(f"[product-id='{target_product_id}']", {}).get("Magento_Swatches/js/swatch-renderer", {}).get("jsonConfig", {}).get("optionPrices", {})

    # Crear una tabla con columnas 'Label', 'Old Price' y 'Final Price'
    column_names2 = ['Label', 'Old Price', 'Final Price']
    table2_data = [{'Label': label, 'Old Price': prices['oldPrice']['amount'], 'Final Price': prices['finalPrice']['amount']} for label, prices in option_prices_data.items()]
    df_table2 = create_table(table2_data, column_names2)

    # Fusionar las tablas en columnas clave comunes
    merged_table = pd.merge(df_table1, df_table2, left_on=['products'], right_on=['Label'], how='left')

    # Crear una nueva columna 'nombre' en merged_table con el mismo valor para todas las filas
    merged_table.insert(0, 'nombre', nombre)

    # Agregar la tabla fusionada a la lista de resultados
    results_list.append(merged_table)

# Concatenar todos los resultados en un solo DataFrame
final_result = pd.concat(results_list, ignore_index=True)

# Eliminar las columnas 'label' y 'products'
final_result = final_result.drop(final_result.columns[[1, 3, 4]], axis=1)

# Imprimir el resultado final
matriz_precios_pikolin = final_result

matriz_precios_pikolin.rename(columns={'label': 'dimension', 'Old Price': 'precio_original', 'Final Price': 'precio_con_descuento'}, inplace=True)

print(matriz_precios_pikolin)

# Subir resultados en Google drive

# Montar Google Drive
# drive.mount('/content/drive', force_remount=True)

# Rutas de los archivos en Google Drive
ruta_archivo_json = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/matriz_pikolin.json'
ruta_archivo_csv = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/matriz_pikolin.csv'

# Guardar el DataFrame en un archivo JSON en Google Drive
#final_result.to_json(ruta_archivo_json, orient='records', lines=True)

# Guardar el DataFrame en un archivo CSV en Google Drive
#final_result.to_csv(ruta_archivo_csv, index=False)



Mounted at /content/drive
             nombre dimension  precio_original  precio_con_descuento
0      Pikolin City    90x190            733.0                 366.5
1      Pikolin City   105x190            813.0                 406.5
2      Pikolin City   135X190            977.0                 488.5
3      Pikolin City   150X190           1115.0                 557.5
4      Pikolin City   150X200           1227.0                 613.5
..              ...       ...              ...                   ...
64  Pikolin Marisma    90x200           1250.0                 562.5
65  Pikolin Marisma   105x200           1390.0                 625.5
66  Pikolin Marisma   160x190           1840.0                 828.0
67  Pikolin Marisma   135X200           1558.0                 701.1
68  Pikolin Marisma   180X200           2182.0                 981.9

[69 rows x 4 columns]


#Emma


In [113]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive', force_remount=True)

# Rutas de los archivos en Google Drive
ruta_archivo_csv = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/url_nombre_a_scrappear.csv'

# Leer el CSV desde Google Drive con especificación de la codificación
df_urls = pd.read_csv(ruta_archivo_csv, encoding='latin1')

# Check for NaN values in the 'URL' column
df_urls = df_urls.dropna(subset=['URL'])

# Filtrar las filas que contienen 'emma-colchon.es' en la columna 'url'
df_urls = df_urls[df_urls['URL'].str.contains('emma-colchon.es')]

# Inicializar una lista para almacenar los DataFrames resultantes
resultados = []

# Iterar sobre cada URL en df_urls
for index, row in df_urls.iterrows():
    url = row['URL']
    nombre = row['Nombre']
    descuento = row['Descuento']

    # Realizar la solicitud HTTP
    response = requests.get(url)

    # Verificar si la solicitud fue exitosa (código 200)
    if response.status_code == 200:
        # Obtener el contenido HTML
        html_content = response.text

        # Crear un objeto BeautifulSoup para analizar el HTML
        soup = BeautifulSoup(html_content, 'html.parser')

        # Encontrar el script con id="structured-data_0"
        target_script = soup.find('script', {'id': 'structured-data_0'})

        # Extraer el contenido del script si se encuentra
        if target_script:
            # Obtener el contenido del script como texto
            script_content = target_script.string

            # Cargar el JSON desde el script
            data = json.loads(script_content)

            # Crear un DataFrame de pandas
            df = pd.DataFrame(data)

            # Seleccionar las columnas 'name' y 'offers'
            df = df[['name', 'offers']]

            # Expandir la columna 'offers' en nuevas columnas
            df = pd.concat([df.drop(['offers'], axis=1), df['offers'].apply(pd.Series)], axis=1)

            # Renombrar columnas'
            df = df.rename(columns={'name': 'lbl'})
            df = df.rename(columns={'price': 'precio_con_descuento'})

            # Seleccionar las columnas finales
            df = df[['lbl', 'precio_con_descuento']]

            # Conservar solo la parte después del último espacio en la columna 'lbl'
            df['lbl'] = df['lbl'].apply(lambda x: x.split(' ')[-1])

            # Añadir las columnas 'nombre' y 'precio original'
            df['nombre'] = nombre
            df['precio_con_descuento'] = pd.to_numeric(df['precio_con_descuento'], errors='coerce')
            descuento_numerico = float(descuento.strip('%')) / 100
            df['precio_original'] = round(df['precio_con_descuento'] / (1 - descuento_numerico), 2) if descuento_numerico != 0 else round(df['price'], 2)

            # Reorganizar las columnas para colocar 'nombre' en la primera posición
            df = df[['nombre', 'lbl', 'precio_original', 'precio_con_descuento']]

            # Agregar el DataFrame resultante a la lista
            resultados.append(df)
        else:
            print(f"No se encontró un script con id='structured-data_0' en la URL: {url}")
    else:
        print(f"Fallo al recuperar la página web. Código de estado: {response.status_code} en la URL: {url}")

# Concatenar todos los DataFrames en uno solo
resultado_final = pd.concat(resultados, ignore_index=True)

# Imprimir el DataFrame final
matriz_precios_emma = resultado_final

matriz_precios_emma.rename(columns={'lbl': 'dimension'}, inplace=True)

print(matriz_precios_emma)

# Rutas de los archivos en Google Drive
ruta_archivo_json = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/matriz_emma.json'
ruta_archivo_csv = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/matriz_emma.csv'

# Guardar el DataFrame en un archivo JSON en Google Drive
# resultado_final.to_json(ruta_archivo_json, orient='records', lines=True)

# Guardar el DataFrame en un archivo CSV en Google Drive
#resultado_final.to_csv(ruta_archivo_csv, index=False)


Mounted at /content/drive
                                 nombre dimension  precio_original  \
0              Almohada Microfibra Emma     40x70            37.37   
1   Almohada Viscoelástica Premium Emma     70x40            99.86   
2                    Base Tapizada Emma    90x190           242.71   
3                    Base Tapizada Emma   135x190           342.71   
4                    Base Tapizada Emma   150x190           385.57   
..                                  ...       ...              ...   
67                   Cama Tapizada Emma   135x190           599.87   
68                   Cama Tapizada Emma   140x200           642.73   
69                   Cama Tapizada Emma   140x200           471.30   
70                   Cama Tapizada Emma   150x200           699.87   
71                   Cama Tapizada Emma   150x200           528.44   

    precio_con_descuento  
0                  29.90  
1                  69.90  
2                 169.90  
3                 239.90 

# Juntando todas las tablas

In [114]:
# Obtén las 4 primeras columnas de cada DataFrame
column_indices = [0, 1, 2, 3]

# Selecciona las columnas deseadas de cada DataFrame
hypnia_selected_columns = matriz_precios_hypnia.iloc[:, column_indices]
pikolin_selected_columns = matriz_precios_pikolin.iloc[:, column_indices]
emma_selected_columns = matriz_precios_emma.iloc[:, column_indices]
ikea_selected_columns = matriz_precios_ikea.iloc[:, column_indices]

# Concatena los DataFrames seleccionados
matriz_precios_scrapping = pd.concat([hypnia_selected_columns, pikolin_selected_columns, emma_selected_columns, ikea_selected_columns], ignore_index=True)

matriz_precios_scrapping = matriz_precios_scrapping.iloc[:, column_indices]

# Asigna nombres a las columnas resultantes
matriz_precios_scrapping.columns = ['nombre', 'dimension', 'precio_original', 'precio_con_descuento']

# Imprimir el resultado
print(matriz_precios_scrapping)

#formato a convertir a json

matriz_precios_prejson = matriz_precios_scrapping.rename(columns={
    'nombre': 'name',
    'dimension': 'var1_item',
    'precio_original': 'price',
    'precio_con_descuento': 'promo'
})
print(matriz_precios_prejson)
# Subir resultados en Google drive

# Rutas de los archivos en Google Drive
ruta_archivo_json = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/matriz_precios_scrapping.json'
ruta_archivo_csv = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/matriz_precios_scrapping.csv'

# Guardar el DataFrame en un archivo JSON en Google Drive
#matriz_precios_scrapping.to_json(ruta_archivo_json, orient='records', lines=True)

# Guardar el DataFrame en un archivo CSV en Google Drive
matriz_precios_scrapping.to_csv(ruta_archivo_csv, index=False)

                            nombre        dimension precio_original  \
0    Almohada Efecto Plumas Hypnia  40 x 70 cm (x2)           99.00   
1       Almohada Ergonómica Hypnia    56 x 36 cm x2          128.00   
2       Almohada Ergonómica Hypnia       56 x 36 cm           70.00   
3          Almohada Luna de Hypnia            50x70           55.00   
4          Almohada Luna de Hypnia            50x70           99.00   
..                             ...              ...             ...   
310                  IKEA Vesteroy       105x190 cm             199   
311                  IKEA Vesteroy       135x190 cm             199   
312                  IKEA Vesteroy       140x200 cm             199   
313                  IKEA Vesteroy       150x190 cm             299   
314                  IKEA Vesteroy       160x200 cm             299   

    precio_con_descuento  
0                  49.00  
1                  89.00  
2                  49.00  
3                  49.00  
4           

#pruebadecolchones.es

In [115]:
from bs4 import BeautifulSoup
import requests as r
import pandas as pd

# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Rutas de los archivos en Google Drive
ruta_archivo_csv = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/url_nombre_a_scrappear.csv'

# Leer el CSV desde Google Drive con especificación de la codificación
df_urls = pd.read_csv(ruta_archivo_csv, encoding='latin1')

# Check for NaN values in the 'URL' column
df_urls = df_urls.dropna(subset=['URL'])

# Filtrar las filas que contienen 'pruebadecolchones.es' en la columna 'url'
df_urls = df_urls[df_urls['URL'].str.contains('pruebadecolchones.es')]

# Crear listas para almacenar los DataFrames individuales
dataframes = []

# Iterar sobre cada fila en el DataFrame df_urls
for index, row in df_urls.iterrows():
    url = row['URL']
    nombre = row['Nombre']

    # Realizar la solicitud y el scraping
    reql = r.get(url)
    soup = BeautifulSoup(reql.text, "html.parser")
    widget = soup.find_all('div', class_='flex flex-col items-center w-1/2 md:w-1/3 my-6 md:my-2 text-center')

    # Crear listas para almacenar los valores de cada columna
    nombres = []
    dimensiones = []
    precios_originales = []
    precios_con_descuento = []

    # Iterar sobre cada elemento en la lista 'widget'
    for element in widget:
        # Para cada elemento, encuentra todos los elementos 'p' y extrae el texto
        prices = element.find_all('p')

        # Realizar limpieza y guardar los valores en las listas correspondientes
        nombre = nombre
        dim = prices[0].text.strip().replace(' cm', '') if prices and len(prices) >= 1 else None
        precios_ori = prices[2].text.strip().replace(' €', '') if prices and len(prices) >= 3 else None
        precios_desc = prices[3].text.strip().replace(' €', '') if prices and len(prices) >= 4 else None

        nombres.append(nombre)
        dimensiones.append(dim)
        precios_originales.append(precios_ori)
        precios_con_descuento.append(precios_desc)

    # Crear un diccionario con las listas
    data = {
        'nombre': nombres,
        'dimension': dimensiones,
        'precio_original': precios_originales,
        'precio_con_descuento': precios_con_descuento
    }

    # Convertir el diccionario en un DataFrame de pandas
    df_resultado = pd.DataFrame(data)

    # Agregar el DataFrame resultante a la lista
    dataframes.append(df_resultado)

# Concatenar todos los DataFrames en uno
matriz_precios_pruebadecolchones = pd.concat(dataframes, ignore_index=True)


# Rellenar el valor de precio_con_descuento con precio_original cuando está vacío
matriz_precios_pruebadecolchones['precio_con_descuento'].fillna(matriz_precios_pruebadecolchones['precio_original'], inplace=True)

# Imprimir el DataFrame final
print(matriz_precios_pruebadecolchones)



# Subir resultados en Google drive

# Rutas de los archivos en Google Drive
ruta_archivo_json = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/matriz_precios_pruebadecolchones.json'
ruta_archivo_csv = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/matriz_precios_pruebadecolchones.csv'

# Guardar el DataFrame en un archivo JSON en Google Drive
#matriz_precios_pruebadecolchones.to_json(ruta_archivo_json, orient='records', lines=True)

# Guardar el DataFrame en un archivo CSV en Google Drive
#matriz_precios_pruebadecolchones.to_csv(ruta_archivo_csv, index=False)

Mounted at /content/drive
                        nombre dimension precio_original precio_con_descuento
0    Hypnia Bienestar Superior    90x190            689               388.55 
1    Hypnia Bienestar Superior    90x200            739               417.05 
2    Hypnia Bienestar Superior   135x190            725               550.05 
3    Hypnia Bienestar Superior  140 x200           1169               664.05 
4    Hypnia Bienestar Superior   150x190           1149               654.55 
..                         ...       ...             ...                  ...
343  colchon dormio supervisco    90x200          87.39                87.39 
344  colchon dormio supervisco   135x190         126.49               126.49 
345  colchon dormio supervisco   135x200         134.99               134.99 
346  colchon dormio supervisco   150x190         140.09               140.09 
347  colchon dormio supervisco   150x200         147.79               147.79 

[348 rows x 4 columns]


#Comparar scrapping con pruebadecolchones.es

In [116]:
import pandas as pd

# Definir los porcentajes de descuento
descuento_hypnia = 0.05  # 5%
descuento_emma = 0.10   # 10%

# Función personalizada para aplicar el descuento según el nombre
def aplicar_descuento(row):
    # Asignar el valor de 'precio_con_descuento' a 'precio_sistema' antes de cualquier transformación
    row['precio_sistema'] = row['precio_con_descuento']

    if 'hypnia' in row['nombre'].lower():
        try:
            # Convertir a float, aplicar el descuento y actualizar el valor
            row['precio_con_descuento'] = float(row['precio_con_descuento']) * (1 - descuento_hypnia)
        except (ValueError, TypeError):
            pass
    elif 'emma' in row['nombre'].lower():
        try:
            # Convertir a float, aplicar el descuento y actualizar el valor
            row['precio_con_descuento'] = float(row['precio_con_descuento']) * (1 - descuento_emma)
        except (ValueError, TypeError):
            pass
    return row

# Aplicar la función a cada fila del DataFrame
matriz_precios_scrapping = matriz_precios_scrapping.apply(aplicar_descuento, axis=1)

# Imprimir el DataFrame resultante
print(matriz_precios_scrapping)

# Guardar el DataFrame en un archivo CSV en local
#matriz_precios_scrapping.to_csv('matriz_precios_scrapping.csv', index=False)


                            nombre        dimension precio_original  \
0    Almohada Efecto Plumas Hypnia  40 x 70 cm (x2)           99.00   
1       Almohada Ergonómica Hypnia    56 x 36 cm x2          128.00   
2       Almohada Ergonómica Hypnia       56 x 36 cm           70.00   
3          Almohada Luna de Hypnia            50x70           55.00   
4          Almohada Luna de Hypnia            50x70           99.00   
..                             ...              ...             ...   
310                  IKEA Vesteroy       105x190 cm             199   
311                  IKEA Vesteroy       135x190 cm             199   
312                  IKEA Vesteroy       140x200 cm             199   
313                  IKEA Vesteroy       150x190 cm             299   
314                  IKEA Vesteroy       160x200 cm             299   

     precio_con_descuento precio_sistema  
0                   46.55          49.00  
1                   84.55          89.00  
2                 

In [117]:

# Unir DataFrames en base a las claves primarias "nombre" y "dimension"
matriz_compare_precios = pd.merge(matriz_precios_pruebadecolchones, matriz_precios_scrapping, on=['nombre', 'dimension'], how='left')

#cambiar nombre columnas
matriz_compare_precios = matriz_compare_precios.rename(columns={
    'nombre': 'nombre',
    'dimension': 'dimension',
    'precio_original_x': 'pruebadecolchones_original',
    'precio_con_descuento_x': 'pruebadecolchones_promo',
    'precio_original_y': 'scrapper_original',
    'precio_con_descuento_y': 'scrapper_promo',
    'precio_sistema': 'precio_para_sistema'
})

# Imprimir el DataFrame resultante
print(matriz_compare_precios)

# Ajustar las listas de orden de columnas según los nombres reales
column_order = ['nombre', 'dimension', 'pruebadecolchones_original', 'pruebadecolchones_promo', 'scrapper_original', 'scrapper_promo', 'precio_para_sistema']
column_order_second = ['nombre', 'dimension', 'pruebadecolchones_original', 'scrapper_original', 'pruebadecolchones_promo', 'scrapper_promo', 'precio_para_sistema']

# Seleccionar solo las columnas necesarias y reordenarlas
matriz_compare_precios = matriz_compare_precios[column_order]

# Cambiar el orden de las columnas según el segundo orden
matriz_compare_precios = matriz_compare_precios[column_order_second]

# Sustituir todos los puntos por comas como separador decimal solo en columnas de tipo objeto
matriz_compare_precios[matriz_compare_precios.select_dtypes(include='object').columns] = matriz_compare_precios.select_dtypes(include='object').apply(lambda x: x.str.replace('.', ','))

print(matriz_compare_precios)

# Subir matriz_compare_precios en Google drive

# Rutas de los archivos en Google Drive
ruta_archivo_json = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/matriz_compare_precios.json'
ruta_archivo_csv = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/matriz_compare_precios.csv'

# Guardar el DataFrame en un archivo CSV en local con punto y coma como separador
matriz_compare_precios.to_csv(ruta_archivo_csv, index=False, sep=';')

print("Tabla guardada")


                        nombre dimension pruebadecolchones_original  \
0    Hypnia Bienestar Superior    90x190                       689    
1    Hypnia Bienestar Superior    90x200                       739    
2    Hypnia Bienestar Superior   135x190                       725    
3    Hypnia Bienestar Superior  140 x200                      1169    
4    Hypnia Bienestar Superior   150x190                      1149    
..                         ...       ...                        ...   
352  colchon dormio supervisco    90x200                     87.39    
353  colchon dormio supervisco   135x190                    126.49    
354  colchon dormio supervisco   135x200                    134.99    
355  colchon dormio supervisco   150x190                    140.09    
356  colchon dormio supervisco   150x200                    147.79    

    pruebadecolchones_promo scrapper_original  scrapper_promo  \
0                   388.55             689.00          388.55   
1                

<ipython-input-117-575bd44221e5>:29: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  matriz_compare_precios[matriz_compare_precios.select_dtypes(include='object').columns] = matriz_compare_precios.select_dtypes(include='object').apply(lambda x: x.str.replace('.', ','))


Original - funciona

In [118]:

# Unir DataFrames en base a las claves primarias "nombre" y "dimension"
matriz_compare_precios = pd.merge(matriz_precios_pruebadecolchones, matriz_precios_scrapping, on=['nombre', 'dimension'], how='left')

# Imprimir el DataFrame resultante
print(matriz_compare_precios)

# Subir matriz_compare_precios en Google drive

# Rutas de los archivos en Google Drive
ruta_archivo_json = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/matriz_compare_precios.json'
ruta_archivo_csv = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/matriz_compare_precios.csv'

# Guardar el DataFrame en un archivo JSON en Google Drive
#matriz_precios_pruebadecolchones.to_json(ruta_archivo_json, orient='records', lines=True)

# Guardar el DataFrame en un archivo CSV en Google Drive
matriz_compare_precios.to_csv(ruta_archivo_csv, index=False)

                        nombre dimension precio_original_x  \
0    Hypnia Bienestar Superior    90x190              689    
1    Hypnia Bienestar Superior    90x200              739    
2    Hypnia Bienestar Superior   135x190              725    
3    Hypnia Bienestar Superior  140 x200             1169    
4    Hypnia Bienestar Superior   150x190             1149    
..                         ...       ...               ...   
352  colchon dormio supervisco    90x200            87.39    
353  colchon dormio supervisco   135x190           126.49    
354  colchon dormio supervisco   135x200           134.99    
355  colchon dormio supervisco   150x190           140.09    
356  colchon dormio supervisco   150x200           147.79    

    precio_con_descuento_x precio_original_y  precio_con_descuento_y  \
0                  388.55             689.00                  388.55   
1                  417.05             739.00                  417.05   
2                  550.05             7

#Exportar a Bubble

# Install packages

In [119]:
# install LINE Python SDK, flask-ngrok
!pip install line-bot-sdk
!pip install pyngrok
[ ]
# install LINE Python SDK, flask-ngrok
!pip install line-bot-sdk
!pip install pyngrok


!pip install request




ERROR: Could not find a version that satisfies the requirement request (from versions: none)
ERROR: No matching distribution found for request


In [120]:
# Import the requests library
import requests

# API endpoint URL
url = "https://app.suggestlab.com/version-test/api/1.1/wf/morganus"

# Making a POST request to the API
response = requests.post(url)

# Check the response
if response.status_code == 200:
    # Processing the JSON response
    data = response.json()
    print("Data retrieved successfully:")
    print(data)
else:
    print("Failed to retrieve data")

JSONDecodeError: ignored

# Connect to LINE bot

In [ ]:
import requests, json

In [ ]:
# แก้ channel_id และ channel_secret
channel_id = "????"
channel_secret = "d0bd9cece91a713f1bc5724d64b72bfe"

In [ ]:
import requests
import json

# Convertir el DataFrame a formato JSON
json_data = matriz_precios_prejson.to_json(orient='records')

# รัน cell เพื่อขอ Short-lived (30 days) channel access token
def issueToken_short(client_id, client_secret):
    endpoint = "https://app.suggestlab.com/version-test/api/1.1/wf/morganus"
    header = {'Content-Type': 'application/json'}
    body = {'grant_type': 'client_credentials',
            'client_id': client_id,
            'client_secret': client_secret}

    try:
        response = requests.post(endpoint, json=body, headers=header)
        response.raise_for_status()

        if response.status_code == 200:
            obj = json.loads(response.text)
            return obj
        else:
            print(f"Error en la solicitud. Código de estado: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error en la solicitud HTTP: {e}")
        return None

tokenDict = issueToken_short(channel_id, channel_secret)
channelAccessToken = tokenDict['access_token']


In [ ]:

# Convertir el DataFrame a formato JSON
json_data = matriz_precios_prejson.to_json(orient='records')

# รัน cell เพื่อขอ Short-lived (30 days) channel access token
def issueToken_short(client_id, client_secret):
  endpoint = "https://app.suggestlab.com/version-test/api/1.1/wf/morganus"
  header = header = {'Content-Type': 'application/json'}
  body = {'grant_type': 'client_credentials',
          'client_id': client_id,
          'client_secret': client_secret}
  response = requests.post(endpoint, json=body, headers=header)
  print(response)
  obj = json.loads(response.text)
  return obj

tokenDict = issueToken_short(channel_id, channel_secret)
channelAccessToken = tokenDict['access_token']
tokenDict

## Create LINE bot object

In [ ]:
from linebot import LineBotApi

# Create app running on localhost (colab) by flask

In [ ]:
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage

In [ ]:
app = Flask(__name__)

handler = WebhookHandler(channel_secret)

@app.route("/webhook.site/5825b18a-f284-475d-9cb2-66d72574b2df", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("# Webhook event:\n", body)
    print('-'*100)

    app.logger.info("Request body: " + body)

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event): # echo function
    print("Message Return\n:", event)
    line_bot_api.reply_message(event.reply_token,
                               TextSendMessage(text=event.message.text))

#print("# Webhook event:\n", body)